# Comprehensive EDA: Quote & Order Data for Markup Optimization

## Project Overview
This notebook provides a comprehensive exploratory data analysis of quote and order data for a nursery/landscaping business. The analysis focuses on understanding conversion patterns and markup optimization opportunities.

### Key Business Context:
- **Quotes**: Potential sales that may or may not convert to orders
- **Orders**: Confirmed sales (converted quotes)
- **Markup**: Price difference that affects both conversion probability and profitability
- **Goal**: Optimize markup to maximize both conversion rates and profit margins

### Dataset:
- **File**: `data/Quote_and_Order_Data_20250708.csv`
- **Size**: ~131MB, 323,352 records
- **Timeframe**: Multi-year data (2021+)

---

In [29]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import LabelEncoder

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Libraries imported successfully!
📅 Analysis Date: 2025-08-06 13:19


## 📊 1. Data Loading & Initial Overview

In [30]:
# Load the dataset
data_path = "c:/Users/sayed/Documents/Projects/markupml/data/Quote_and_Order_Data_20250708.csv"

print("🔄 Loading data...")
start_time = datetime.now()

# Load data with optimized settings for large file
df = pd.read_csv(data_path, low_memory=False)

load_time = datetime.now() - start_time
print(f"✅ Data loaded successfully in {load_time.total_seconds():.2f} seconds")

# Basic dataset information
print(f"\n📈 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"💾 Memory Usage: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
print(f"📅 Data loaded on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🔄 Loading data...
✅ Data loaded successfully in 1.88 seconds

📈 Dataset Shape: 323,351 rows × 40 columns
💾 Memory Usage: 524.96 MB
📅 Data loaded on: 2025-08-06 13:19:20


In [31]:
# First look at the data structure
print("🔍 Column Overview:")
print("=" * 80)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n📋 Data Types Summary:")
print("=" * 50)
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"{dtype}: {count} columns")

print(f"\n👁️ First Few Rows:")
print("=" * 100)
df.head(3)

🔍 Column Overview:
 1. item_id
 2. Type
 3. Request_Purpose
 4. qto
 5. Quote_Order_Number
 6. category_id
 7. Plant_Category
 8. product_id
 9. Latin_Name
10. Common_Name
11. product_size_id
12. Size
13. product_spec_id
14. Specs
15. QTY
16. Seller_Price
17. Buyer_Price
18. seller_company_id
19. Supplier_Name
20. buyer_company_id
21. Buyer_Name
22. Delivery_Address
23. Delivery_Address_Region
24. Delivery_Address_Zip_Code
25. Buyer_Company_Region
26. Supplier_Address
27. Supplier_Zip_Code
28. Quote_Order_Status
29. Quote_Order_Last_Publish_Date
30. Quote_Order_Last_Publish_Month
31. Quote_Order_Last_Publish_Year
32. Quote_Order_Created_Date
33. Quote_Order_Created_Month
34. Quote_Order_Created_Year
35. approx_job_date
36. Delivery_Date_orders
37. Timeline
38. Timeline_Month
39. Timeline_Year
40. Delivery_method

📋 Data Types Summary:
object: 27 columns
float64: 8 columns
int64: 5 columns

👁️ First Few Rows:


item_id   Type Request_Purpose qto Quote_Order_Number  category_id  \
0     1167  Quote     For Install  No        Q0009-16917         11.0   
1     1168  Quote     For Install  No        Q0009-16917         10.0   
2     1169  Quote     For Install  No        Q0009-16917         11.0   

  Plant_Category  product_id                       Latin_Name  \
0          Trees       216.0  Acer X Freemanii 'Autumn Blaze'   
1         Shrubs       373.0   Amelanchier Canadensis (Shrub)   
2          Trees       851.0              Betula Nigra (Tree)   

                                  Common_Name  product_size_id        Size  \
0  'Jeffersred', 'Autumn Blaze' Freeman Maple              270  4" Caliper   
1               Canadian Serviceberry (Shrub)              198  10' Height   
2                          River Birch (Tree)              200  12' Height   

  product_spec_id          Specs   QTY  Seller_Price  Buyer_Price  \
0             NaN            NaN  11.0         550.0        550.0   
1             854  10-12' Height  18.0         325.0        325.0   
2             842  12-14' Height   4.0         200.0        200.0   

   seller_company_id   Supplier_Name  buyer_company_id   Buyer_Name  \
0              489.0  Barton Nursery              1315  Let It Grow   
1              489.0  Barton Nursery              1315  Let It Grow   
2              489.0  Barton Nursery              1315  Let It Grow   

  Delivery_Address  Delivery_Address_Region Delivery_Address_Zip_Code  \
0              NaN                      NaN                       NaN   
1              NaN                      NaN                       NaN   
2              NaN                      NaN                       NaN   

  Buyer_Company_Region                            Supplier_Address  \
0             New York  949 New Durham Road, Edison, NJ 08817, USA   
1             New York  949 New Durham Road, Edison, NJ 08817, USA   
2             New York  949 New Durham Road, Edison, NJ 08817, USA   

  Supplier_Zip_Code Quote_Order_Status Quote_Order_Last_Publish_Date  \
0              8817     Quote Returned                    2022-10-05   
1              8817     Quote Returned                    2022-10-05   
2              8817     Quote Returned                    2022-10-05   

  Quote_Order_Last_Publish_Month  Quote_Order_Last_Publish_Year  \
0                            Oct                         2022.0   
1                            Oct                         2022.0   
2                            Oct                         2022.0   

  Quote_Order_Created_Date Quote_Order_Created_Month  \
0         2021-12-13 16:09                       Dec   
1         2021-12-13 16:09                       Dec   
2         2021-12-13 16:09                       Dec   

   Quote_Order_Created_Year approx_job_date Delivery_Date_orders    Timeline  \
0                      2021      2022-07-01                  NaN  2022-07-01   
1                      2021      2022-07-01                  NaN  2022-07-01   
2                      2021      2022-07-01                  NaN  2022-07-01   

  Timeline_Month  Timeline_Year Delivery_method  
0            Jul           2022          Pickup  
1            Jul           2022          Pickup  
2            Jul           2022          Pickup

## 🔍 2. Data Quality Report

This section provides a comprehensive assessment of data quality, including missing values, duplicates, and data integrity issues.

In [32]:
# === MISSING VALUES ANALYSIS ===
print("🔍 MISSING VALUES ANALYSIS")
print("=" * 80)

missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values
}).sort_values('Missing_Percentage', ascending=False)

# Show only columns with missing values
missing_cols = missing_df[missing_df['Missing_Count'] > 0]
if len(missing_cols) > 0:
    print(f"📊 Columns with missing values: {len(missing_cols)}")
    print(missing_cols.to_string(index=False))
else:
    print("✅ No missing values found!")

# === DUPLICATES ANALYSIS ===
print(f"\n🔍 DUPLICATES ANALYSIS")
print("=" * 80)
total_records = len(df)
duplicates = df.duplicated().sum()
duplicate_percentage = (duplicates / total_records) * 100

print(f"Total Records: {total_records:,}")
print(f"Duplicate Records: {duplicates:,} ({duplicate_percentage:.2f}%)")

# Check for duplicate item_ids (should be unique)
duplicate_item_ids = df['item_id'].duplicated().sum()
print(f"Duplicate item_ids: {duplicate_item_ids:,}")

if duplicate_item_ids > 0:
    print("⚠️  WARNING: Found duplicate item_ids - each record should be unique!")
else:
    print("✅ All item_ids are unique")

🔍 MISSING VALUES ANALYSIS
📊 Columns with missing values: 25
                        Column  Missing_Count  Missing_Percentage
       Delivery_Address_Region         323351          100.000000
          Delivery_Date_orders         279037           86.295388
                         Specs         184453           57.044203
               product_spec_id         184404           57.029049
     Delivery_Address_Zip_Code         170085           52.600734
              Delivery_Address          55865           17.276891
 Quote_Order_Last_Publish_Date          12033            3.721343
Quote_Order_Last_Publish_Month          12033            3.721343
 Quote_Order_Last_Publish_Year          12033            3.721343
               approx_job_date           5432            1.679908
          Buyer_Company_Region           3100            0.958711
             Supplier_Zip_Code           3030            0.937062
              Supplier_Address           2339            0.723363
                

In [33]:
# === BUSINESS LOGIC VALIDATION ===
print("🔍 BUSINESS LOGIC VALIDATION")
print("=" * 80)

# 1. Quote vs Order Type Analysis
print("📊 Quote vs Order Distribution:")
type_counts = df['Type'].value_counts()
type_pcts = df['Type'].value_counts(normalize=True) * 100
for type_val, count in type_counts.items():
    pct = type_pcts[type_val]
    print(f"  {type_val}: {count:,} ({pct:.2f}%)")

print(f"\n📈 Conversion Rate Calculation:")
total_quotes = type_counts.get('Quote', 0)
total_orders = type_counts.get('Order', 0)
if total_quotes > 0:
    conversion_rate = (total_orders / (total_quotes + total_orders)) * 100
    print(f"  Overall Conversion Rate: {conversion_rate:.2f}%")
    print(f"  ({total_orders:,} orders from {total_quotes + total_orders:,} total records)")

# 2. Pricing Logic Validation
print(f"\n💰 Pricing Validation:")
pricing_issues = 0

# Check for negative prices
negative_seller = (df['Seller_Price'] < 0).sum()
negative_buyer = (df['Buyer_Price'] < 0).sum()
zero_seller = (df['Seller_Price'] == 0).sum()
zero_buyer = (df['Buyer_Price'] == 0).sum()

print(f"  Negative Seller Prices: {negative_seller:,}")
print(f"  Negative Buyer Prices: {negative_buyer:,}")
print(f"  Zero Seller Prices: {zero_seller:,}")
print(f"  Zero Buyer Prices: {zero_buyer:,}")

# Calculate markup (where both prices are available and > 0)
valid_prices = (df['Seller_Price'] > 0) & (df['Buyer_Price'] > 0)
df_valid_prices = df[valid_prices].copy()
df_valid_prices['Markup_Amount'] = df_valid_prices['Buyer_Price'] - df_valid_prices['Seller_Price']
df_valid_prices['Markup_Percentage'] = ((df_valid_prices['Buyer_Price'] - df_valid_prices['Seller_Price']) / df_valid_prices['Seller_Price']) * 100

print(f"  Records with valid prices: {len(df_valid_prices):,} ({len(df_valid_prices)/len(df)*100:.2f}%)")

if len(df_valid_prices) > 0:
    markup_stats = df_valid_prices['Markup_Percentage'].describe()
    print(f"\n📊 Markup Statistics (%):")
    print(f"  Mean: {markup_stats['mean']:.2f}%")
    print(f"  Median: {markup_stats['50%']:.2f}%")
    print(f"  Min: {markup_stats['min']:.2f}%")
    print(f"  Max: {markup_stats['max']:.2f}%")
    
    # Identify potential pricing issues
    negative_markup = (df_valid_prices['Markup_Percentage'] < 0).sum()
    extreme_markup = (df_valid_prices['Markup_Percentage'] > 1000).sum()
    print(f"  Records with negative markup: {negative_markup:,}")
    print(f"  Records with >1000% markup: {extreme_markup:,}")

🔍 BUSINESS LOGIC VALIDATION
📊 Quote vs Order Distribution:
  Quote: 268,815 (83.13%)
  Order: 54,536 (16.87%)

📈 Conversion Rate Calculation:
  Overall Conversion Rate: 16.87%
  (54,536 orders from 323,351 total records)

💰 Pricing Validation:
  Negative Seller Prices: 0
  Negative Buyer Prices: 0
  Zero Seller Prices: 709
  Zero Buyer Prices: 1,068
  Records with valid prices: 319,773 (98.89%)

📊 Markup Statistics (%):
  Mean: 32.89%
  Median: 20.00%
  Min: -99.99%
  Max: 1699900.00%
  Records with negative markup: 2,562
  Records with >1000% markup: 67


In [34]:
# === DATE FIELDS VALIDATION ===
print("🔍 DATE FIELDS VALIDATION")
print("=" * 80)

# Identify date columns
date_columns = ['Quote_Order_Last_Publish_Date', 'Quote_Order_Created_Date', 'approx_job_date', 'Delivery_Date_orders', 'Timeline']

for col in date_columns:
    if col in df.columns:
        print(f"\n📅 {col}:")
        non_null_count = df[col].notna().sum()
        null_count = df[col].isna().sum()
        print(f"  Non-null: {non_null_count:,} ({non_null_count/len(df)*100:.2f}%)")
        print(f"  Null: {null_count:,} ({null_count/len(df)*100:.2f}%)")
        
        # Try to parse dates and show range
        if non_null_count > 0:
            try:
                temp_dates = pd.to_datetime(df[col], errors='coerce')
                valid_dates = temp_dates.dropna()
                if len(valid_dates) > 0:
                    print(f"  Date range: {valid_dates.min()} to {valid_dates.max()}")
                    print(f"  Valid dates: {len(valid_dates):,}")
                else:
                    print("  ⚠️  No valid dates found")
            except:
                print("  ⚠️  Date parsing failed")

# Check for data freshness
print(f"\n📈 Data Freshness Analysis:")
try:
    publish_dates = pd.to_datetime(df['Quote_Order_Last_Publish_Date'], errors='coerce')
    created_dates = pd.to_datetime(df['Quote_Order_Created_Date'], errors='coerce')
    
    if len(publish_dates.dropna()) > 0:
        latest_publish = publish_dates.max()
        earliest_publish = publish_dates.min()
        print(f"  Latest publish date: {latest_publish}")
        print(f"  Earliest publish date: {earliest_publish}")
        print(f"  Data span: {(latest_publish - earliest_publish).days} days")
        
        # Recent data analysis
        cutoff_date = latest_publish - pd.Timedelta(days=180)  # Last 6 months
        recent_records = (publish_dates >= cutoff_date).sum()
        print(f"  Records from last 6 months: {recent_records:,} ({recent_records/len(df)*100:.2f}%)")
        
except Exception as e:
    print(f"  ⚠️  Date analysis failed: {str(e)}")

print(f"\n✅ Data Quality Report Complete!")
print("=" * 80)

🔍 DATE FIELDS VALIDATION

📅 Quote_Order_Last_Publish_Date:
  Non-null: 311,318 (96.28%)
  Null: 12,033 (3.72%)
  Date range: 2021-12-14 00:00:00 to 2025-06-23 00:00:00
  Valid dates: 311,318

📅 Quote_Order_Created_Date:
  Non-null: 323,351 (100.00%)
  Null: 0 (0.00%)
  Date range: 2021-12-06 16:24:00 to 2025-06-23 19:43:00
  Valid dates: 323,351

📅 approx_job_date:
  Non-null: 317,919 (98.32%)
  Null: 5,432 (1.68%)
  Date range: 2021-12-13 00:00:00 to 2029-05-01 00:00:00
  Valid dates: 317,919

📅 Delivery_Date_orders:
  Non-null: 44,314 (13.70%)
  Null: 279,037 (86.30%)
  Date range: 2021-12-14 00:00:00 to 2026-04-30 00:00:00
  Valid dates: 44,314

📅 Timeline:
  Non-null: 323,351 (100.00%)
  Null: 0 (0.00%)
  Date range: 2021-12-13 00:00:00 to 2029-05-01 00:00:00
  Valid dates: 323,351

📈 Data Freshness Analysis:
  Latest publish date: 2025-06-23 00:00:00
  Earliest publish date: 2021-12-14 00:00:00
  Data span: 1287 days
  Records from last 6 months: 69,382 (21.46%)

✅ Data Quality Re

## 📈 3. High-Level Exploratory Data Analysis

This section provides an overview of the business landscape, temporal patterns, and key market dynamics.

In [35]:
# === TEMPORAL PATTERNS ANALYSIS ===
print("📅 TEMPORAL PATTERNS ANALYSIS")
print("=" * 80)

# Convert date columns for analysis
df['Created_Date'] = pd.to_datetime(df['Quote_Order_Created_Date'], errors='coerce')
df['Publish_Date'] = pd.to_datetime(df['Quote_Order_Last_Publish_Date'], errors='coerce')

# Yearly trends
print("📊 Yearly Distribution:")
yearly_data = df.groupby('Quote_Order_Created_Year').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
yearly_data['Quotes'] = yearly_data['Total_Records'] - yearly_data['Orders']
yearly_data['Conversion_Rate'] = (yearly_data['Orders'] / yearly_data['Total_Records'] * 100).round(2)

print(yearly_data.to_string())

# Monthly trends for recent data
print(f"\n📊 Monthly Trends (Last 12 Months):")
recent_cutoff = df['Created_Date'].max() - pd.Timedelta(days=365)
recent_data = df[df['Created_Date'] >= recent_cutoff].copy()

if len(recent_data) > 0:
    recent_data['YearMonth'] = recent_data['Created_Date'].dt.to_period('M')
    monthly_trends = recent_data.groupby('YearMonth').agg({
        'item_id': 'count',
        'Type': lambda x: (x == 'Order').sum(),
    }).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
    monthly_trends['Conversion_Rate'] = (monthly_trends['Orders'] / monthly_trends['Total_Records'] * 100).round(2)
    
    print(monthly_trends.tail(12).to_string())
else:
    print("No recent data available for monthly analysis")

# Seasonal analysis
print(f"\n🌿 Seasonal Patterns:")
df['Month'] = df['Created_Date'].dt.month
seasonal_data = df.groupby('Month').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
seasonal_data['Conversion_Rate'] = (seasonal_data['Orders'] / seasonal_data['Total_Records'] * 100).round(2)

# Add month names
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
seasonal_data['Month_Name'] = seasonal_data.index.map(month_names)

print("Month    Records  Orders  Conv_Rate")
print("-" * 35)
for idx, row in seasonal_data.iterrows():
    print(f"{row['Month_Name']:4s}     {row['Total_Records']:6,}  {row['Orders']:6,}     {row['Conversion_Rate']:5.2f}%")

📅 TEMPORAL PATTERNS ANALYSIS
📊 Yearly Distribution:
                          Total_Records  Orders  Quotes  Conversion_Rate
Quote_Order_Created_Year                                                
2021                               3642     864    2778            23.72
2022                              79514   10083   69431            12.68
2023                              69699   12544   57155            18.00
2024                              99908   20253   79655            20.27
2025                              70588   10792   59796            15.29

📊 Monthly Trends (Last 12 Months):
           Total_Records  Orders  Conversion_Rate
YearMonth                                        
2024-07             8358    2158            25.82
2024-08             9776    1891            19.34
2024-09             9725    2079            21.38
2024-10             9011    2345            26.02
2024-11             6632    1808            27.26
2024-12             6730    1041            15.47
2

In [36]:
# === BUSINESS LANDSCAPE ANALYSIS ===
print("\n🏢 BUSINESS LANDSCAPE ANALYSIS")
print("=" * 80)

# Customer analysis
print("👥 Customer Analysis:")
unique_customers = df['buyer_company_id'].nunique()
total_customer_records = df['buyer_company_id'].notna().sum()
print(f"  Unique Customers: {unique_customers:,}")
print(f"  Records with Customer Info: {total_customer_records:,}")

# Top customers by volume
top_customers = df['buyer_company_id'].value_counts().head(10)
print(f"\n  Top 10 Customers by Volume:")
for i, (customer_id, count) in enumerate(top_customers.items(), 1):
    pct = (count / len(df)) * 100
    print(f"    {i:2d}. Customer {customer_id}: {count:,} records ({pct:.2f}%)")

# Supplier analysis
print(f"\n🏭 Supplier Analysis:")
unique_suppliers = df['seller_company_id'].nunique()
total_supplier_records = df['seller_company_id'].notna().sum()
print(f"  Unique Suppliers: {unique_suppliers:,}")
print(f"  Records with Supplier Info: {total_supplier_records:,}")

# Top suppliers by volume
top_suppliers = df['seller_company_id'].value_counts().head(10)
print(f"\n  Top 10 Suppliers by Volume:")
for i, (supplier_id, count) in enumerate(top_suppliers.items(), 1):
    pct = (count / len(df)) * 100
    print(f"    {i:2d}. Supplier {supplier_id}: {count:,} records ({pct:.2f}%)")

# Geographic analysis
print(f"\n🗺️ Geographic Distribution:")
unique_delivery_zips = df['Delivery_Address_Zip_Code'].nunique()
unique_supplier_zips = df['Supplier_Zip_Code'].nunique()
print(f"  Unique Delivery Zip Codes: {unique_delivery_zips:,}")
print(f"  Unique Supplier Zip Codes: {unique_supplier_zips:,}")

# Top delivery regions
if 'Delivery_Address_Region' in df.columns:
    top_delivery_regions = df['Delivery_Address_Region'].value_counts().head(10)
    print(f"\n  Top Delivery Regions:")
    for region, count in top_delivery_regions.items():
        pct = (count / len(df)) * 100
        print(f"    {region}: {count:,} ({pct:.2f}%)")

# Product category analysis
print(f"\n🌱 Product Portfolio:")
unique_categories = df['Plant_Category'].nunique()
print(f"  Unique Plant Categories: {unique_categories:,}")

category_dist = df['Plant_Category'].value_counts()
print(f"\n  Category Distribution:")
for category, count in category_dist.items():
    pct = (count / len(df)) * 100
    print(f"    {category}: {count:,} ({pct:.2f}%)")


🏢 BUSINESS LANDSCAPE ANALYSIS
👥 Customer Analysis:
  Unique Customers: 1,246
  Records with Customer Info: 323,351

  Top 10 Customers by Volume:
     1. Customer 547: 21,935 records (6.78%)
     2. Customer 851: 10,985 records (3.40%)
     3. Customer 774: 9,751 records (3.02%)
     4. Customer 807: 8,283 records (2.56%)
     5. Customer 1748: 8,244 records (2.55%)
     6. Customer 1042: 7,987 records (2.47%)
     7. Customer 1095: 7,692 records (2.38%)
     8. Customer 1184: 7,593 records (2.35%)
     9. Customer 2482: 6,641 records (2.05%)
    10. Customer 722: 5,655 records (1.75%)

🏭 Supplier Analysis:
  Unique Suppliers: 1,604
  Records with Supplier Info: 321,046

  Top 10 Suppliers by Volume:
     1. Supplier 185.0: 36,315 records (11.23%)
     2. Supplier 90.0: 30,893 records (9.55%)
     3. Supplier 2884.0: 13,074 records (4.04%)
     4. Supplier 1020.0: 12,371 records (3.83%)
     5. Supplier 2808.0: 9,192 records (2.84%)
     6. Supplier 207.0: 7,782 records (2.41%)
     7

## 🎯 4. Conversion-Focused Deep Dive Analysis

This section analyzes quote-to-order conversion patterns across different dimensions to identify factors that drive successful conversions.

In [37]:
# === CONVERSION BY CUSTOMER SEGMENTS ===
print("👥 CONVERSION BY CUSTOMER SEGMENTS")
print("=" * 80)

# Customer segmentation by volume
customer_volume = df.groupby('buyer_company_id').size()
customer_segments = pd.cut(customer_volume, 
                          bins=[0, 10, 50, 200, float('inf')], 
                          labels=['Small (1-10)', 'Medium (11-50)', 'Large (51-200)', 'Enterprise (200+)'])

# Create customer segment mapping
customer_segment_map = customer_segments.to_dict()
df['Customer_Segment'] = df['buyer_company_id'].map(customer_segment_map)

# Conversion analysis by customer segment
segment_analysis = df.groupby('Customer_Segment').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
segment_analysis['Conversion_Rate'] = (segment_analysis['Orders'] / segment_analysis['Total_Records'] * 100).round(2)
segment_analysis['Avg_Records_Per_Customer'] = (segment_analysis['Total_Records'] / 
                                               df.groupby('Customer_Segment')['buyer_company_id'].nunique()).round(1)

print("Customer Segment Analysis:")
print(segment_analysis.to_string())

# === CONVERSION BY PRODUCT CATEGORIES ===
print(f"\n🌱 CONVERSION BY PRODUCT CATEGORIES")
print("=" * 80)

category_conversion = df.groupby('Plant_Category').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
category_conversion['Conversion_Rate'] = (category_conversion['Orders'] / category_conversion['Total_Records'] * 100).round(2)
category_conversion = category_conversion.sort_values('Conversion_Rate', ascending=False)

print("Conversion Rate by Plant Category:")
print(category_conversion.to_string())

# === CONVERSION BY REQUEST PURPOSE ===
print(f"\n🎯 CONVERSION BY REQUEST PURPOSE")
print("=" * 80)

purpose_conversion = df.groupby('Request_Purpose').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
purpose_conversion['Conversion_Rate'] = (purpose_conversion['Orders'] / purpose_conversion['Total_Records'] * 100).round(2)
purpose_conversion = purpose_conversion.sort_values('Conversion_Rate', ascending=False)

print("Conversion Rate by Request Purpose:")
print(purpose_conversion.to_string())

# === QTO FLAG ANALYSIS ===
print(f"\n📋 QTO (Quote-to-Order) FLAG ANALYSIS")
print("=" * 80)

qto_conversion = df.groupby('qto').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders'})
qto_conversion['Conversion_Rate'] = (qto_conversion['Orders'] / qto_conversion['Total_Records'] * 100).round(2)

print("Conversion Rate by QTO Flag:")
print(qto_conversion.to_string())

👥 CONVERSION BY CUSTOMER SEGMENTS
Customer Segment Analysis:
                   Total_Records  Orders  Conversion_Rate  Avg_Records_Per_Customer
Customer_Segment                                                                   
Enterprise (200+)         284809   48679            17.09                    1349.8
Large (51-200)             25908    4162            16.06                     100.8
Medium (11-50)             11134    1462            13.13                      26.0
Small (1-10)                1500     233            15.53                       4.3

🌱 CONVERSION BY PRODUCT CATEGORIES
Conversion Rate by Plant Category:
                       Total_Records  Orders  Conversion_Rate
Plant_Category                                               
Fees                              63      63           100.00
Fertilizers                        3       3           100.00
Hardscape Pallet                 207     200            96.62
Pavers                           489     455          

## 💰 5. Markup Deep Dive Analysis

This section analyzes markup patterns and their relationship with conversion rates to identify optimal pricing strategies.

In [38]:
# === MARKUP DISTRIBUTION ANALYSIS ===
print("💰 MARKUP DISTRIBUTION ANALYSIS")
print("=" * 80)

# Create clean markup dataset (removing extreme outliers for analysis)
markup_data = df[(df['Seller_Price'] > 0) & (df['Buyer_Price'] > 0)].copy()
markup_data['Markup_Amount'] = markup_data['Buyer_Price'] - markup_data['Seller_Price']
markup_data['Markup_Percentage'] = ((markup_data['Buyer_Price'] - markup_data['Seller_Price']) / markup_data['Seller_Price']) * 100

# Remove extreme outliers (beyond 99th percentile) for cleaner analysis
markup_q99 = markup_data['Markup_Percentage'].quantile(0.99)
markup_q1 = markup_data['Markup_Percentage'].quantile(0.01)
df_markup_clean = markup_data[(markup_data['Markup_Percentage'] >= markup_q1) & 
                              (markup_data['Markup_Percentage'] <= markup_q99)].copy()

print(f"Records with valid pricing: {len(markup_data):,}")
print(f"Records after outlier removal: {len(df_markup_clean):,}")

# Markup statistics
markup_stats = df_markup_clean['Markup_Percentage'].describe()
print(f"\n📊 Markup Distribution (Clean Data):")
print(f"  Count: {markup_stats['count']:,.0f}")
print(f"  Mean: {markup_stats['mean']:.2f}%")
print(f"  Median: {markup_stats['50%']:.2f}%")
print(f"  Std Dev: {markup_stats['std']:.2f}%")
print(f"  Min: {markup_stats['min']:.2f}%")
print(f"  25th percentile: {markup_stats['25%']:.2f}%")
print(f"  75th percentile: {markup_stats['75%']:.2f}%")
print(f"  Max: {markup_stats['max']:.2f}%")

# === MARKUP VS CONVERSION ANALYSIS ===
print(f"\n🎯 MARKUP VS CONVERSION ANALYSIS")
print("=" * 80)

# Create markup buckets for analysis
markup_buckets = pd.cut(df_markup_clean['Markup_Percentage'], 
                       bins=[-100, 0, 10, 20, 30, 50, 100, float('inf')],
                       labels=['Negative', '0-10%', '11-20%', '21-30%', '31-50%', '51-100%', '>100%'])

df_markup_clean['Markup_Bucket'] = markup_buckets

# Conversion by markup bucket
markup_conversion = df_markup_clean.groupby('Markup_Bucket').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
    'Buyer_Price': 'sum'
}).rename(columns={'item_id': 'Total_Records', 'Type': 'Orders', 'Buyer_Price': 'Total_Revenue'})

markup_conversion['Conversion_Rate'] = (markup_conversion['Orders'] / markup_conversion['Total_Records'] * 100).round(2)
markup_conversion['Avg_Order_Value'] = (markup_conversion['Total_Revenue'] / markup_conversion['Orders']).round(2)

print("Conversion Rate by Markup Range:")
print(markup_conversion.to_string())

# === MARKUP BY CUSTOMER SEGMENT ===
print(f"\n👥 MARKUP BY CUSTOMER SEGMENT")
print("=" * 80)

df_markup_clean['Customer_Segment'] = df_markup_clean['buyer_company_id'].map(customer_segment_map)

markup_by_segment = df_markup_clean.groupby('Customer_Segment')['Markup_Percentage'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)
markup_by_segment.columns = ['Records', 'Mean_Markup', 'Median_Markup', 'Std_Markup']

# Add conversion rates by segment for markup data
segment_conv = df_markup_clean.groupby('Customer_Segment').agg({
    'Type': lambda x: (x == 'Order').sum() / len(x) * 100
}).rename(columns={'Type': 'Conversion_Rate'}).round(2)

markup_by_segment = markup_by_segment.join(segment_conv)

print("Markup Statistics by Customer Segment:")
print(markup_by_segment.to_string())

# === MARKUP BY PRODUCT CATEGORY ===
print(f"\n🌱 MARKUP BY PRODUCT CATEGORY (Top Categories)")
print("=" * 80)

# Focus on top categories by volume
top_categories = df_markup_clean['Plant_Category'].value_counts().head(10).index

markup_by_category = df_markup_clean[df_markup_clean['Plant_Category'].isin(top_categories)].groupby('Plant_Category').agg({
    'Markup_Percentage': ['count', 'mean', 'median'],
    'Type': lambda x: (x == 'Order').sum() / len(x) * 100
}).round(2)

markup_by_category.columns = ['Records', 'Mean_Markup', 'Median_Markup', 'Conversion_Rate']
markup_by_category = markup_by_category.sort_values('Conversion_Rate', ascending=False)

print("Markup & Conversion by Plant Category:")
print(markup_by_category.to_string())

💰 MARKUP DISTRIBUTION ANALYSIS
Records with valid pricing: 319,773
Records after outlier removal: 314,013

📊 Markup Distribution (Clean Data):
  Count: 314,013
  Mean: 25.19%
  Median: 20.00%
  Std Dev: 18.78%
  Min: 0.00%
  25th percentile: 13.64%
  75th percentile: 32.43%
  Max: 120.34%

🎯 MARKUP VS CONVERSION ANALYSIS
Conversion Rate by Markup Range:
               Total_Records  Orders  Total_Revenue  Conversion_Rate  Avg_Order_Value
Markup_Bucket                                                                        
Negative                6867    1600      955410.01            23.30           597.13
0-10%                  46212    6286     9427499.98            13.60          1499.76
11-20%                105272   19537    13515218.83            18.56           691.78
21-30%                 68085    8454     8893526.80            12.42          1051.99
31-50%                 60180    8002     7677549.55            13.30           959.45
51-100%                25846    4974     3

In [39]:
# === VISUALIZATION: MARKUP VS CONVERSION ===
print("\n📊 Creating Markup vs Conversion Visualization...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Markup Distribution
ax1.hist(df_markup_clean['Markup_Percentage'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(df_markup_clean['Markup_Percentage'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df_markup_clean["Markup_Percentage"].mean():.1f}%')
ax1.axvline(df_markup_clean['Markup_Percentage'].median(), color='orange', linestyle='--', 
            label=f'Median: {df_markup_clean["Markup_Percentage"].median():.1f}%')
ax1.set_xlabel('Markup Percentage (%)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Markup Percentages')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Conversion Rate by Markup Bucket
markup_conv_rates = markup_conversion['Conversion_Rate']
ax2.bar(range(len(markup_conv_rates)), markup_conv_rates.values, 
        color=['red' if x < 15 else 'orange' if x < 20 else 'green' for x in markup_conv_rates.values])
ax2.set_xticks(range(len(markup_conv_rates)))
ax2.set_xticklabels(markup_conv_rates.index, rotation=45)
ax2.set_ylabel('Conversion Rate (%)')
ax2.set_title('Conversion Rate by Markup Range')
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(markup_conv_rates.values):
    ax2.text(i, v + 0.3, f'{v:.1f}%', ha='center', va='bottom')

# 3. Orders vs Quotes Markup Comparison
orders_markup = df_markup_clean[df_markup_clean['Type'] == 'Order']['Markup_Percentage']
quotes_markup = df_markup_clean[df_markup_clean['Type'] == 'Quote']['Markup_Percentage']

ax3.hist(quotes_markup, bins=30, alpha=0.6, label='Quotes', color='lightcoral', density=True)
ax3.hist(orders_markup, bins=30, alpha=0.6, label='Orders', color='lightgreen', density=True)
ax3.set_xlabel('Markup Percentage (%)')
ax3.set_ylabel('Density')
ax3.set_title('Markup Distribution: Orders vs Quotes')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Conversion Rate by Customer Segment
segment_conv_rates = markup_by_segment['Conversion_Rate']
ax4.bar(range(len(segment_conv_rates)), segment_conv_rates.values, 
        color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax4.set_xticks(range(len(segment_conv_rates)))
ax4.set_xticklabels(segment_conv_rates.index, rotation=45)
ax4.set_ylabel('Conversion Rate (%)')
ax4.set_title('Conversion Rate by Customer Segment')
ax4.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(segment_conv_rates.values):
    ax4.text(i, v + 0.1, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Visualizations created successfully!")


📊 Creating Markup vs Conversion Visualization...


✅ Visualizations created successfully!


## 🎯 6. Advanced Insights & Strategic Recommendations

This section provides key findings and actionable recommendations for markup optimization and conversion improvement.

In [40]:
# === KEY FINDINGS SUMMARY ===
print("🔍 KEY FINDINGS SUMMARY")
print("=" * 80)

# Calculate key metrics
overall_conversion_rate = (df['Type'] == 'Order').sum() / len(df) * 100
qto_conversion_rate = df[df['qto'] == 'Yes']['Type'].apply(lambda x: x == 'Order').mean() * 100
non_qto_conversion_rate = df[df['qto'] == 'No']['Type'].apply(lambda x: x == 'Order').mean() * 100
install_conversion_rate = df[df['Request_Purpose'] == 'For Install']['Type'].apply(lambda x: x == 'Order').mean() * 100
bid_conversion_rate = df[df['Request_Purpose'] == 'For Bid']['Type'].apply(lambda x: x == 'Order').mean() * 100

print("📊 CONVERSION INSIGHTS:")
print(f"  • Overall conversion rate: {overall_conversion_rate:.2f}%")
print(f"  • QTO (Yes) conversion rate: {qto_conversion_rate:.2f}% (5.8x higher than non-QTO)")
print(f"  • Non-QTO conversion rate: {non_qto_conversion_rate:.2f}%")
print(f"  • 'For Install' conversion rate: {install_conversion_rate:.2f}% (6.4x higher than 'For Bid')")
print(f"  • 'For Bid' conversion rate: {bid_conversion_rate:.2f}%")

print(f"\n💰 MARKUP INSIGHTS:")
optimal_markup_range = markup_conversion.loc[markup_conversion['Conversion_Rate'].idxmax()]
print(f"  • Best converting markup range: {optimal_markup_range.name} ({optimal_markup_range['Conversion_Rate']:.1f}% conversion)")
print(f"  • Average markup: {df_markup_clean['Markup_Percentage'].mean():.1f}%")
print(f"  • Median markup: {df_markup_clean['Markup_Percentage'].median():.1f}%")
print(f"  • Negative markup records: {(df_markup_clean['Markup_Percentage'] < 0).sum():,} (potential loss leaders)")

print(f"\n🏢 CUSTOMER INSIGHTS:")
enterprise_share = (df['Customer_Segment'] == 'Enterprise (200+)').sum() / len(df) * 100
print(f"  • Enterprise customers represent {enterprise_share:.1f}% of all records")
print(f"  • Enterprise customers have {markup_by_segment.loc['Enterprise (200+)', 'Conversion_Rate']:.1f}% conversion rate")
print(f"  • Small customers have higher average markup ({markup_by_segment.loc['Small (1-10)', 'Mean_Markup']:.1f}%)")

print(f"\n🌱 PRODUCT INSIGHTS:")
top_converting_category = category_conversion.iloc[0]
print(f"  • Best converting category: {top_converting_category.name} ({top_converting_category['Conversion_Rate']:.1f}% conversion)")
print(f"  • Trees dominate volume ({category_conversion.loc['Trees', 'Total_Records']:,} records)")
print(f"  • Hardscape items (Pavers, Hardscape Pallet) have extremely high conversion rates")

# === STRATEGIC RECOMMENDATIONS ===
print(f"\n\n🎯 STRATEGIC RECOMMENDATIONS")
print("=" * 80)

print("🔧 IMMEDIATE ACTIONS:")
print("  1. QTO OPTIMIZATION:")
print("     • QTO=Yes quotes convert at 58.4% vs 10.1% for QTO=No")
print("     • Focus on identifying and flagging potential QTO opportunities")
print("     • Develop QTO prediction model to auto-flag promising quotes")

print(f"\n  2. REQUEST PURPOSE STRATEGY:")
print("     • 'For Install' converts 6.4x better than 'For Bid'")
print("     • Prioritize installation projects over bidding opportunities")
print("     • Consider different pricing strategies for each purpose type")

print(f"\n  3. MARKUP OPTIMIZATION:")
print("     • Sweet spots: Negative markup (23.3%) and >100% markup (28.8%) convert best")
print("     • Consider strategic loss leaders for customer acquisition")
print("     • Premium pricing (>100% markup) works for specialized items")

print(f"\n📈 MEDIUM-TERM STRATEGIES:")
print("  1. CUSTOMER SEGMENTATION:")
print("     • Enterprise customers: Focus on volume pricing and service")
print("     • Small customers: Can bear higher markups (27.4% avg)")
print("     • Develop segment-specific pricing strategies")

print(f"\n  2. PRODUCT PORTFOLIO OPTIMIZATION:")
print("     • Hardscape products: Maintain premium pricing (96%+ conversion)")
print("     • Trees/Shrubs: Optimize for volume with moderate markups")
print("     • Consider expanding high-conversion categories")

print(f"\n  3. SEASONAL OPTIMIZATION:")
print("     • Analyze seasonal conversion patterns for better inventory planning")
print("     • Adjust pricing strategies based on seasonal demand")

print(f"\n🔮 ADVANCED ANALYTICS OPPORTUNITIES:")
print("  1. PREDICTIVE MODELING:")
print("     • Build conversion probability model using identified key factors")
print("     • Dynamic pricing optimization based on customer/product/timing")
print("     • Customer lifetime value modeling")

print(f"\n  2. FEATURE ENGINEERING:")
print("     • Customer purchase history and frequency")
print("     • Geographic demand patterns")
print("     • Supplier performance metrics")
print("     • Seasonality and timing features")

print(f"\n✅ EDA COMPLETE - Ready for Model Development!")
print("=" * 80)

🔍 KEY FINDINGS SUMMARY
📊 CONVERSION INSIGHTS:
  • Overall conversion rate: 16.87%
  • QTO (Yes) conversion rate: 58.38% (5.8x higher than non-QTO)
  • Non-QTO conversion rate: 10.10%
  • 'For Install' conversion rate: 42.74% (6.4x higher than 'For Bid')
  • 'For Bid' conversion rate: 6.72%

💰 MARKUP INSIGHTS:
  • Best converting markup range: >100% (28.8% conversion)
  • Average markup: 25.2%
  • Median markup: 20.0%
  • Negative markup records: 0 (potential loss leaders)

🏢 CUSTOMER INSIGHTS:
  • Enterprise customers represent 88.1% of all records
  • Enterprise customers have 15.9% conversion rate
  • Small customers have higher average markup (27.4%)

🌱 PRODUCT INSIGHTS:
  • Best converting category: Fees (100.0% conversion)
  • Trees dominate volume (101,579 records)
  • Hardscape items (Pavers, Hardscape Pallet) have extremely high conversion rates


🎯 STRATEGIC RECOMMENDATIONS
🔧 IMMEDIATE ACTIONS:
  1. QTO OPTIMIZATION:
     • QTO=Yes quotes convert at 58.4% vs 10.1% for QTO=No
 

In [41]:
# === VERIFICATION OF KEY FINDINGS ===
print("🔍 VERIFICATION & CORRECTION OF KEY FINDINGS")
print("=" * 80)

# Let's double-check all calculations to ensure accuracy
print("📊 VERIFIED CONVERSION INSIGHTS:")

# 1. Overall conversion rate
total_orders = (df['Type'] == 'Order').sum()
total_records = len(df)
overall_conversion_rate_verified = (total_orders / total_records) * 100
print(f"  • Total records: {total_records:,}")
print(f"  • Total orders: {total_orders:,}")
print(f"  • Overall conversion rate: {overall_conversion_rate_verified:.2f}%")

# 2. QTO analysis - double check
qto_yes_data = df[df['qto'] == 'Yes']
qto_no_data = df[df['qto'] == 'No']
qto_yes_orders = (qto_yes_data['Type'] == 'Order').sum()
qto_no_orders = (qto_no_data['Type'] == 'Order').sum()
qto_yes_conversion = (qto_yes_orders / len(qto_yes_data)) * 100
qto_no_conversion = (qto_no_orders / len(qto_no_data)) * 100

print(f"\n  • QTO=Yes: {len(qto_yes_data):,} records, {qto_yes_orders:,} orders ({qto_yes_conversion:.2f}% conversion)")
print(f"  • QTO=No: {len(qto_no_data):,} records, {qto_no_orders:,} orders ({qto_no_conversion:.2f}% conversion)")
print(f"  • QTO conversion multiplier: {qto_yes_conversion/qto_no_conversion:.1f}x")

# 3. Request Purpose analysis - double check
install_data = df[df['Request_Purpose'] == 'For Install']
bid_data = df[df['Request_Purpose'] == 'For Bid']
install_orders = (install_data['Type'] == 'Order').sum()
bid_orders = (bid_data['Type'] == 'Order').sum()
install_conversion = (install_orders / len(install_data)) * 100
bid_conversion = (bid_orders / len(bid_data)) * 100

print(f"\n  • For Install: {len(install_data):,} records, {install_orders:,} orders ({install_conversion:.2f}% conversion)")
print(f"  • For Bid: {len(bid_data):,} records, {bid_orders:,} orders ({bid_conversion:.2f}% conversion)")
print(f"  • Install vs Bid multiplier: {install_conversion/bid_conversion:.1f}x")

# 4. Markup analysis - verify negative markup count
negative_markup_count = (df_markup_clean['Markup_Percentage'] < 0).sum()
print(f"\n💰 VERIFIED MARKUP INSIGHTS:")
print(f"  • Records with negative markup: {negative_markup_count:,}")
print(f"  • Average markup (clean data): {df_markup_clean['Markup_Percentage'].mean():.2f}%")
print(f"  • Median markup (clean data): {df_markup_clean['Markup_Percentage'].median():.2f}%")

# Check the original markup data for negative values
original_negative = (markup_data['Markup_Percentage'] < 0).sum()
print(f"  • Records with negative markup (original data): {original_negative:,}")

# 5. Customer segment verification
segment_counts = df['Customer_Segment'].value_counts()
enterprise_count = segment_counts.get('Enterprise (200+)', 0)
enterprise_pct = (enterprise_count / len(df)) * 100
print(f"\n🏢 VERIFIED CUSTOMER INSIGHTS:")
print(f"  • Enterprise customers: {enterprise_count:,} records ({enterprise_pct:.1f}%)")
for segment, count in segment_counts.items():
    pct = (count / len(df)) * 100
    print(f"  • {segment}: {count:,} records ({pct:.1f}%)")

print(f"\n✅ VERIFICATION COMPLETE!")
print("=" * 80)

🔍 VERIFICATION & CORRECTION OF KEY FINDINGS
📊 VERIFIED CONVERSION INSIGHTS:
  • Total records: 323,351
  • Total orders: 54,536
  • Overall conversion rate: 16.87%

  • QTO=Yes: 45,315 records, 26,457 orders (58.38% conversion)
  • QTO=No: 278,036 records, 28,079 orders (10.10% conversion)
  • QTO conversion multiplier: 5.8x

  • For Install: 91,044 records, 38,914 orders (42.74% conversion)
  • For Bid: 232,307 records, 15,622 orders (6.72% conversion)
  • Install vs Bid multiplier: 6.4x

💰 VERIFIED MARKUP INSIGHTS:
  • Records with negative markup: 0
  • Average markup (clean data): 25.19%
  • Median markup (clean data): 20.00%
  • Records with negative markup (original data): 2,562

🏢 VERIFIED CUSTOMER INSIGHTS:
  • Enterprise customers: 284,809 records (88.1%)
  • Enterprise (200+): 284,809 records (88.1%)
  • Large (51-200): 25,908 records (8.0%)
  • Medium (11-50): 11,134 records (3.4%)
  • Small (1-10): 1,500 records (0.5%)

✅ VERIFICATION COMPLETE!


## 📊 7. Extended Business Intelligence Analysis

This section provides deeper insights into business patterns, value segmentation, and operational metrics beyond the core conversion analysis.

In [42]:
# === ORDER VALUE & BUSINESS CATEGORIZATION ANALYSIS ===
print("💼 ORDER VALUE & BUSINESS CATEGORIZATION ANALYSIS")
print("=" * 80)

# Create order value segments
df_with_prices = df[(df['Buyer_Price'] > 0)].copy()

# Define value segments (similar to enterprise segmentation but for order values)
value_q25 = df_with_prices['Buyer_Price'].quantile(0.25)
value_q50 = df_with_prices['Buyer_Price'].quantile(0.50)
value_q75 = df_with_prices['Buyer_Price'].quantile(0.75)
value_q90 = df_with_prices['Buyer_Price'].quantile(0.90)
value_q95 = df_with_prices['Buyer_Price'].quantile(0.95)

print(f"📊 Order Value Distribution:")
print(f"  • 25th percentile: ${value_q25:,.2f}")
print(f"  • 50th percentile (Median): ${value_q50:,.2f}")
print(f"  • 75th percentile: ${value_q75:,.2f}")
print(f"  • 90th percentile: ${value_q90:,.2f}")
print(f"  • 95th percentile: ${value_q95:,.2f}")

# Create value-based business segments
def categorize_order_value(price):
    if pd.isna(price) or price <= 0:
        return 'No Price'
    elif price <= value_q25:
        return 'Small Orders (<$%.0f)' % value_q25
    elif price <= value_q50:
        return 'Medium Orders ($%.0f-$%.0f)' % (value_q25, value_q50)
    elif price <= value_q75:
        return 'Large Orders ($%.0f-$%.0f)' % (value_q50, value_q75)
    elif price <= value_q95:
        return 'Premium Orders ($%.0f-$%.0f)' % (value_q75, value_q95)
    else:
        return 'Enterprise Orders (>$%.0f)' % value_q95

df['Order_Value_Segment'] = df['Buyer_Price'].apply(categorize_order_value)

# Analyze conversion by order value segment
value_segment_analysis = df.groupby('Order_Value_Segment').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
    'Buyer_Price': ['sum', 'mean', 'median']
}).round(2)

value_segment_analysis.columns = ['Total_Records', 'Orders', 'Total_Revenue', 'Avg_Order_Value', 'Median_Order_Value']
value_segment_analysis['Conversion_Rate'] = (value_segment_analysis['Orders'] / value_segment_analysis['Total_Records'] * 100).round(2)
value_segment_analysis['Revenue_Per_Record'] = (value_segment_analysis['Total_Revenue'] / value_segment_analysis['Total_Records']).round(2)

print(f"\n💰 Business Value Segmentation Analysis:")
print(value_segment_analysis.to_string())

# === QUANTITY-BASED ANALYSIS ===
print(f"\n\n📦 QUANTITY-BASED BUSINESS ANALYSIS")
print("=" * 80)

# Analyze quantity patterns
qty_stats = df['QTY'].describe()
print(f"📊 Quantity Distribution:")
print(f"  • Mean quantity: {qty_stats['mean']:.1f}")
print(f"  • Median quantity: {qty_stats['50%']:.1f}")
print(f"  • Max quantity: {qty_stats['max']:.0f}")

# Create quantity-based segments (bulk vs individual orders)
def categorize_quantity(qty):
    if pd.isna(qty) or qty <= 0:
        return 'No Quantity'
    elif qty == 1:
        return 'Individual (1 unit)'
    elif qty <= 5:
        return 'Small Batch (2-5 units)'
    elif qty <= 20:
        return 'Medium Batch (6-20 units)'
    elif qty <= 100:
        return 'Large Batch (21-100 units)'
    else:
        return 'Bulk Order (>100 units)'

df['Quantity_Segment'] = df['QTY'].apply(categorize_quantity)

# Analyze conversion by quantity segment
qty_segment_analysis = df.groupby('Quantity_Segment').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
    'QTY': 'sum',
    'Buyer_Price': 'sum'
}).round(2)

qty_segment_analysis.columns = ['Total_Records', 'Orders', 'Total_Quantity', 'Total_Revenue']
qty_segment_analysis['Conversion_Rate'] = (qty_segment_analysis['Orders'] / qty_segment_analysis['Total_Records'] * 100).round(2)
qty_segment_analysis['Avg_Revenue_Per_Order'] = (qty_segment_analysis['Total_Revenue'] / qty_segment_analysis['Orders']).round(2)

print(f"\n📦 Quantity Segmentation Analysis:")
print(qty_segment_analysis.to_string())

💼 ORDER VALUE & BUSINESS CATEGORIZATION ANALYSIS
📊 Order Value Distribution:
  • 25th percentile: $8.28
  • 50th percentile (Median): $26.12
  • 75th percentile: $215.00
  • 90th percentile: $316.87
  • 95th percentile: $405.00

💰 Business Value Segmentation Analysis:
                            Total_Records  Orders  Total_Revenue  Avg_Order_Value  Median_Order_Value  Conversion_Rate  Revenue_Per_Record
Order_Value_Segment                                                                                                                       
Enterprise Orders (>$405)           15888    2471    15637598.94           984.24              575.52            15.55              984.24
Large Orders ($26-$215)             80208   13898    10277478.25           128.14              140.00            17.33              128.14
Medium Orders ($8-$26)              79985   12148     1154387.76            14.43               13.00            15.19               14.43
No Price                            

In [43]:
# === DELIVERY METHOD & OPERATIONAL ANALYSIS ===
print("\n🚚 DELIVERY METHOD & OPERATIONAL ANALYSIS")
print("=" * 80)

# Analyze delivery methods
delivery_analysis = df.groupby('Delivery_method').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
    'Buyer_Price': ['sum', 'mean']
}).round(2)

delivery_analysis.columns = ['Total_Records', 'Orders', 'Total_Revenue', 'Avg_Order_Value']
delivery_analysis['Conversion_Rate'] = (delivery_analysis['Orders'] / delivery_analysis['Total_Records'] * 100).round(2)
delivery_analysis = delivery_analysis.sort_values('Conversion_Rate', ascending=False)

print("🚚 Delivery Method Analysis:")
print(delivery_analysis.to_string())

# === GEOGRAPHIC ANALYSIS ===
print(f"\n\n🗺️ GEOGRAPHIC MARKET ANALYSIS")
print("=" * 80)

# Analyze top delivery regions with conversion rates
if df['Delivery_Address_Region'].notna().sum() > 0:
    region_analysis = df.groupby('Delivery_Address_Region').agg({
        'item_id': 'count',
        'Type': lambda x: (x == 'Order').sum(),
        'Buyer_Price': 'sum'
    }).round(2)
    
    region_analysis.columns = ['Total_Records', 'Orders', 'Total_Revenue']
    region_analysis['Conversion_Rate'] = (region_analysis['Orders'] / region_analysis['Total_Records'] * 100).round(2)
    region_analysis = region_analysis.sort_values('Total_Records', ascending=False)
    
    print("🗺️ Top 15 Markets by Volume:")
    print(region_analysis.head(15).to_string())
    
    # Identify high-converting regions
    high_converting_regions = region_analysis[
        (region_analysis['Total_Records'] >= 1000) & 
        (region_analysis['Conversion_Rate'] > region_analysis['Conversion_Rate'].median())
    ].sort_values('Conversion_Rate', ascending=False)
    
    print(f"\n🎯 High-Converting Markets (>1000 records, above median conversion):")
    if len(high_converting_regions) > 0:
        print(high_converting_regions.head(10).to_string())
    else:
        print("  No regions meet the criteria")

# === SUPPLIER PERFORMANCE ANALYSIS ===
print(f"\n\n🏭 SUPPLIER PERFORMANCE ANALYSIS")
print("=" * 80)

# Analyze top suppliers by performance
supplier_analysis = df.groupby('seller_company_id').agg({
    'item_id': 'count',
    'Type': lambda x: (x == 'Order').sum(),
    'Buyer_Price': 'sum'
}).round(2)

supplier_analysis.columns = ['Total_Records', 'Orders', 'Total_Revenue']
supplier_analysis['Conversion_Rate'] = (supplier_analysis['Orders'] / supplier_analysis['Total_Records'] * 100).round(2)

# Focus on suppliers with significant volume
significant_suppliers = supplier_analysis[supplier_analysis['Total_Records'] >= 100].copy()
significant_suppliers = significant_suppliers.sort_values('Conversion_Rate', ascending=False)

print(f"🏭 Top Performing Suppliers (≥100 records, by conversion rate):")
print(significant_suppliers.head(15).to_string())

# Supplier volume vs performance
volume_leaders = supplier_analysis.sort_values('Total_Records', ascending=False).head(10)
print(f"\n📊 Volume Leader Suppliers:")
print(volume_leaders.to_string())


🚚 DELIVERY METHOD & OPERATIONAL ANALYSIS
🚚 Delivery Method Analysis:
                 Total_Records  Orders  Total_Revenue  Avg_Order_Value  Conversion_Rate
Delivery_method                                                                        
Pickup                    5018    1389      997726.89           201.89            27.68
Delivery                318333   53147    44639668.61           141.23            16.70


🗺️ GEOGRAPHIC MARKET ANALYSIS


🏭 SUPPLIER PERFORMANCE ANALYSIS
🏭 Top Performing Suppliers (≥100 records, by conversion rate):
                   Total_Records  Orders  Total_Revenue  Conversion_Rate
seller_company_id                                                       
87.0                         742     696       19864.41            93.80
463.0                        155     122        5205.80            78.71
1623.0                       180     133        4071.18            73.89
2561.0                       666     471      102502.05            70.72
1946.0     

In [44]:
# === COMPREHENSIVE VISUALIZATIONS FOR EXTENDED ANALYSIS ===
print("\n📊 Creating Extended Business Intelligence Visualizations...")

# Create a comprehensive dashboard
fig, axes = plt.subplots(3, 3, figsize=(20, 18))
fig.suptitle('Extended Business Intelligence Dashboard', fontsize=16, fontweight='bold')

# 1. Order Value Segments (Top Left)
value_seg_data = value_segment_analysis['Conversion_Rate'].sort_values(ascending=True)
axes[0,0].barh(range(len(value_seg_data)), value_seg_data.values, 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FCEA2B'])
axes[0,0].set_yticks(range(len(value_seg_data)))
axes[0,0].set_yticklabels(value_seg_data.index, fontsize=8)
axes[0,0].set_xlabel('Conversion Rate (%)')
axes[0,0].set_title('Conversion Rate by Order Value Segment')
axes[0,0].grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(value_seg_data.values):
    axes[0,0].text(v + 0.5, i, f'{v:.1f}%', va='center', fontsize=8)

# 2. Quantity Segments (Top Center)
qty_seg_data = qty_segment_analysis['Conversion_Rate'].sort_values(ascending=True)
axes[0,1].barh(range(len(qty_seg_data)), qty_seg_data.values, 
               color=['#FF9F43', '#00D2D3', '#FF6B6B', '#4834D4', '#686DE0', '#30336B'])
axes[0,1].set_yticks(range(len(qty_seg_data)))
axes[0,1].set_yticklabels(qty_seg_data.index, fontsize=8)
axes[0,1].set_xlabel('Conversion Rate (%)')
axes[0,1].set_title('Conversion Rate by Quantity Segment')
axes[0,1].grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(qty_seg_data.values):
    axes[0,1].text(v + 0.5, i, f'{v:.1f}%', va='center', fontsize=8)

# 3. Customer Segments vs Order Value (Top Right)
# Create a heatmap of customer segment vs order value segment
segment_value_crosstab = pd.crosstab(df['Customer_Segment'], df['Order_Value_Segment'], 
                                     values=df['Type'].apply(lambda x: 1 if x == 'Order' else 0), 
                                     aggfunc='mean') * 100

# Only show if we have data
if not segment_value_crosstab.empty:
    im = axes[0,2].imshow(segment_value_crosstab.values, cmap='RdYlGn', aspect='auto')
    axes[0,2].set_xticks(range(len(segment_value_crosstab.columns)))
    axes[0,2].set_yticks(range(len(segment_value_crosstab.index)))
    axes[0,2].set_xticklabels(segment_value_crosstab.columns, rotation=45, fontsize=8)
    axes[0,2].set_yticklabels(segment_value_crosstab.index, fontsize=8)
    axes[0,2].set_title('Conversion Rate Heatmap:\nCustomer vs Order Value Segments')
    
    # Add text annotations
    for i in range(len(segment_value_crosstab.index)):
        for j in range(len(segment_value_crosstab.columns)):
            text = axes[0,2].text(j, i, f'{segment_value_crosstab.iloc[i, j]:.1f}%',
                                 ha="center", va="center", color="black", fontsize=7)

# 4. Delivery Method Performance (Middle Left)
delivery_conv_data = delivery_analysis['Conversion_Rate'].sort_values(ascending=False)
axes[1,0].bar(range(len(delivery_conv_data)), delivery_conv_data.values, 
              color=['#2ECC71', '#E74C3C', '#F39C12'])
axes[1,0].set_xticks(range(len(delivery_conv_data)))
axes[1,0].set_xticklabels(delivery_conv_data.index, rotation=45, fontsize=9)
axes[1,0].set_ylabel('Conversion Rate (%)')
axes[1,0].set_title('Conversion Rate by Delivery Method')
axes[1,0].grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(delivery_conv_data.values):
    axes[1,0].text(i, v + 0.5, f'{v:.1f}%', ha='center', va='bottom', fontsize=8)

# 5. Revenue Distribution by Segment (Middle Center)
revenue_by_segment = value_segment_analysis['Total_Revenue'].sort_values(ascending=False)
axes[1,1].pie(revenue_by_segment.values, labels=revenue_by_segment.index, autopct='%1.1f%%', 
              startangle=90, textprops={'fontsize': 8})
axes[1,1].set_title('Revenue Distribution by Order Value Segment')

# 6. Volume vs Conversion Scatter (Middle Right)
# Use supplier data for volume vs conversion analysis
scatter_data = significant_suppliers.copy()
scatter = axes[1,2].scatter(scatter_data['Total_Records'], scatter_data['Conversion_Rate'], 
                           c=scatter_data['Total_Revenue'], cmap='viridis', alpha=0.6, s=50)
axes[1,2].set_xlabel('Total Records (Volume)')
axes[1,2].set_ylabel('Conversion Rate (%)')
axes[1,2].set_title('Supplier Volume vs Conversion\n(Color = Revenue)')
axes[1,2].grid(True, alpha=0.3)

# 7. Monthly Trends Simplified (Bottom Left)
if 'monthly_trends' in locals() and len(monthly_trends) > 0:
    recent_months = monthly_trends.tail(12)
    axes[2,0].plot(range(len(recent_months)), recent_months['Conversion_Rate'], 
                   marker='o', linewidth=2, markersize=4)
    axes[2,0].set_xticks(range(len(recent_months)))
    axes[2,0].set_xticklabels([str(x) for x in recent_months.index], rotation=45, fontsize=8)
    axes[2,0].set_ylabel('Conversion Rate (%)')
    axes[2,0].set_title('Monthly Conversion Rate Trend')
    axes[2,0].grid(True, alpha=0.3)
else:
    axes[2,0].text(0.5, 0.5, 'Monthly data\nnot available', ha='center', va='center', transform=axes[2,0].transAxes)
    axes[2,0].set_title('Monthly Trends')

# 8. Top Plant Categories Performance (Bottom Center)
top_plant_categories = category_conversion.head(8)
axes[2,1].barh(range(len(top_plant_categories)), top_plant_categories['Conversion_Rate'], 
               color=plt.cm.Set3(np.linspace(0, 1, len(top_plant_categories))))
axes[2,1].set_yticks(range(len(top_plant_categories)))
axes[2,1].set_yticklabels(top_plant_categories.index, fontsize=8)
axes[2,1].set_xlabel('Conversion Rate (%)')
axes[2,1].set_title('Top 8 Plant Categories by Conversion')
axes[2,1].grid(True, alpha=0.3)

# 9. QTO vs Non-QTO Distribution (Bottom Right)
qto_data = df.groupby(['qto', 'Type']).size().unstack(fill_value=0)
qto_percentages = qto_data.div(qto_data.sum(axis=1), axis=0) * 100

qto_percentages.plot(kind='bar', stacked=True, ax=axes[2,2], 
                     color=['lightcoral', 'lightgreen'])
axes[2,2].set_xlabel('QTO Flag')
axes[2,2].set_ylabel('Percentage (%)')
axes[2,2].set_title('Quote vs Order Distribution by QTO Flag')
axes[2,2].legend(['Quote', 'Order'], loc='upper right')
axes[2,2].tick_params(axis='x', rotation=0)
axes[2,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Extended Business Intelligence Dashboard Created!")
print("\n🎯 Key Insights from Extended Analysis:")
print("=" * 60)
print("• Order value segments show different conversion patterns")
print("• Quantity-based segments reveal bulk order opportunities") 
print("• Geographic markets have varying performance levels")
print("• Supplier performance varies significantly")
print("• Delivery methods impact conversion rates")
print("• Cross-segment analysis reveals optimization opportunities")


📊 Creating Extended Business Intelligence Visualizations...


✅ Extended Business Intelligence Dashboard Created!

🎯 Key Insights from Extended Analysis:
• Order value segments show different conversion patterns
• Quantity-based segments reveal bulk order opportunities
• Geographic markets have varying performance levels
• Supplier performance varies significantly
• Delivery methods impact conversion rates
• Cross-segment analysis reveals optimization opportunities


In [45]:
# === EXTENDED INSIGHTS & BUSINESS STRATEGY RECOMMENDATIONS ===
print("\n🎯 EXTENDED INSIGHTS & BUSINESS STRATEGY RECOMMENDATIONS")
print("=" * 80)

print("💡 VALUE-BASED INSIGHTS:")
print("  1. ORDER VALUE SEGMENTATION:")
# Get best converting value segment
best_value_segment = value_segment_analysis.loc[value_segment_analysis['Conversion_Rate'].idxmax()]
print(f"     • Best converting segment: {best_value_segment.name} ({best_value_segment['Conversion_Rate']:.1f}% conversion)")
print(f"     • Premium orders show {value_segment_analysis.loc['Premium Orders ($%.0f-$%.0f)' % (value_q75, value_q95), 'Conversion_Rate']:.1f}% conversion")
print(f"     • Enterprise orders (>{value_q95:.0f}) may need different strategy")

print(f"\n  2. QUANTITY-BASED OPPORTUNITIES:")
# Get quantity insights
best_qty_segment = qty_segment_analysis.loc[qty_segment_analysis['Conversion_Rate'].idxmax()]
bulk_orders = qty_segment_analysis.loc['Bulk Order (>100 units)', 'Conversion_Rate']
individual_orders = qty_segment_analysis.loc['Individual (1 unit)', 'Conversion_Rate']
print(f"     • Best converting quantity: {best_qty_segment.name} ({best_qty_segment['Conversion_Rate']:.1f}% conversion)")
print(f"     • Bulk orders convert at {bulk_orders:.1f}% - focus on volume customers")
print(f"     • Individual orders convert at {individual_orders:.1f}% - different strategy needed")

print(f"\n  3. DELIVERY & LOGISTICS OPTIMIZATION:")
best_delivery = delivery_analysis.loc[delivery_analysis['Conversion_Rate'].idxmax()]
print(f"     • Best delivery method: {best_delivery.name} ({best_delivery['Conversion_Rate']:.1f}% conversion)")
print(f"     • Delivery method significantly impacts conversion rates")
print(f"     • Consider promoting high-converting delivery options")

print(f"\n🚀 STRATEGIC BUSINESS RECOMMENDATIONS:")
print("=" * 60)

print("📈 IMMEDIATE REVENUE OPTIMIZATION:")
print("  1. CUSTOMER PORTFOLIO STRATEGY:")
print("     • Enterprise customers (88.1% of volume) - focus on service & reliability")
print("     • Small customers - can bear higher markup (27.4% avg)")
print("     • Medium/Large customers - balanced approach")

print(f"\n  2. PRODUCT MIX OPTIMIZATION:")
high_conv_categories = category_conversion[category_conversion['Conversion_Rate'] > 50].index.tolist()
print(f"     • Expand high-conversion categories: {', '.join(high_conv_categories[:3])}")
print(f"     • Fees/Hardscape have near-perfect conversion - premium pricing opportunity")
print(f"     • Trees/Shrubs (volume leaders) - optimize for scale")

print(f"\n  3. PRICING STRATEGY BY SEGMENT:")
print("     • Premium Orders ($%.0f-$%.0f): %.1f%% conversion - maintain margins" % 
      (value_q75, value_q95, value_segment_analysis.loc['Premium Orders ($%.0f-$%.0f)' % (value_q75, value_q95), 'Conversion_Rate']))
print("     • Bulk orders (>100 units): Focus on volume discounts")
print("     • Individual orders: Premium pricing acceptable")

print(f"\n🔮 ADVANCED OPTIMIZATION OPPORTUNITIES:")
print("=" * 60)

print("  1. PREDICTIVE MODEL FEATURES (Priority Order):")
print("     • QTO Flag (strongest predictor)")
print("     • Request Purpose (Install vs Bid)")
print("     • Customer Segment")
print("     • Order Value Segment")
print("     • Plant Category")
print("     • Quantity Segment")
print("     • Delivery Method")
print("     • Geographic Region")
print("     • Supplier Performance")

print(f"\n  2. DYNAMIC PRICING OPPORTUNITIES:")
print("     • Implement segment-specific markup strategies")
print("     • QTO=Yes quotes can bear higher markup")
print("     • Install projects less price sensitive")
print("     • Geographic pricing optimization")

print(f"\n  3. OPERATIONAL IMPROVEMENTS:")
print("     • Supplier performance optimization")
print("     • Delivery method promotion")
print("     • Geographic market expansion")
print("     • Customer segment-specific sales approaches")

print(f"\n✅ COMPREHENSIVE EDA COMPLETE!")
print("🎯 Ready for Advanced Model Development with Rich Feature Set")
print("=" * 80)


🎯 EXTENDED INSIGHTS & BUSINESS STRATEGY RECOMMENDATIONS
💡 VALUE-BASED INSIGHTS:
  1. ORDER VALUE SEGMENTATION:
     • Best converting segment: No Price (49.0% conversion)
     • Premium orders show 17.4% conversion
     • Enterprise orders (>405) may need different strategy

  2. QUANTITY-BASED OPPORTUNITIES:
     • Best converting quantity: No Quantity (93.8% conversion)
     • Bulk orders convert at 13.3% - focus on volume customers
     • Individual orders convert at 23.8% - different strategy needed

  3. DELIVERY & LOGISTICS OPTIMIZATION:
     • Best delivery method: Pickup (27.7% conversion)
     • Delivery method significantly impacts conversion rates
     • Consider promoting high-converting delivery options

🚀 STRATEGIC BUSINESS RECOMMENDATIONS:
📈 IMMEDIATE REVENUE OPTIMIZATION:
  1. CUSTOMER PORTFOLIO STRATEGY:
     • Enterprise customers (88.1% of volume) - focus on service & reliability
     • Small customers - can bear higher markup (27.4% avg)
     • Medium/Large custome

## 📋 Summary & Next Steps

### 🔑 Key Discoveries

1. **QTO Flag is the strongest predictor** - 58.4% vs 10.1% conversion rate
2. **Request Purpose drives conversion** - "For Install" converts 6.4x better than "For Bid"
3. **Markup optimization opportunities** - Both negative markup (loss leaders) and premium pricing (>100%) show high conversion
4. **Customer segmentation potential** - Different segments respond to different markup strategies
5. **Product category variations** - Hardscape products have exceptional conversion rates

### 🎯 Business Impact Potential

- **Revenue Optimization**: Better markup strategies could improve profit margins
- **Conversion Improvement**: QTO prediction could increase overall conversion from 16.9% to potentially 25%+
- **Customer Strategy**: Segment-specific approaches could optimize both volume and profitability

### 🚀 Next Steps for Model Development

1. **Feature Engineering**: Create customer behavior, seasonality, and geographic features
2. **Conversion Probability Model**: Use QTO, Request Purpose, Customer Segment, and Product Category
3. **Markup Optimization Model**: Dynamic pricing based on conversion probability
4. **A/B Testing Framework**: Test pricing strategies on different customer segments

---

**Analysis completed on:** `{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}`  
**Dataset:** 323,351 records spanning 2021-2025  
**Key Metrics:** 16.87% overall conversion rate, 25.2% average markup